In [49]:
import pandas as pd
import re
import string
import sklearn

In [50]:
df = pd.read_csv("synopsis2.csv")

In [51]:
print(df.head())

                                             Comment  Y
0  In the opening, Clint Barton is teaching his d...  1
1  Five years later, Scott Lang escapes from the ...  1
2  With Stark now on board with the plan, the rem...  1
3  After testing the quantum time machine on Bart...  1
4  Rocket and Thor travel to Asgard to retrieve t...  1


In [52]:
len(df.index)

7

In [53]:
index=[]
for i in range(len(df.index)):
    index.append("Comment "+str(i))
#df.insert(loc=0,column='index',value=index)
df['index']=index
df.set_index('index', inplace=True)
df    

,Comment,Y
index,,
Comment 0,"In the opening, Clint Barton is teaching his d...",1
Comment 1,"Five years later, Scott Lang escapes from the ...",1
Comment 2,"With Stark now on board with the plan, the rem...",1
Comment 3,After testing the quantum time machine on Bart...,1
Comment 4,Rocket and Thor travel to Asgard to retrieve t...,1
Comment 5,After everyone has returned to the present on ...,1
Comment 6,"Following the battle, The Avengers hold a fune...",1


In [54]:
print(index)

['Comment 0', 'Comment 1', 'Comment 2', 'Comment 3', 'Comment 4', 'Comment 5', 'Comment 6']


In [55]:
def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text
round1 = lambda x: clean_text_round1(x)

In [56]:
data_clean = pd.DataFrame(df.Comment.apply(round1))
data_clean

,Comment
index,
Comment 0,in the opening clint barton is teaching his da...
Comment 1,five years later scott lang escapes from the q...
Comment 2,with stark now on board with the plan the rema...
Comment 3,after testing the quantum time machine on bart...
Comment 4,rocket and thor travel to asgard to retrieve t...
Comment 5,after everyone has returned to the present on ...
Comment 6,following the battle the avengers hold a funer...


In [57]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_clean.Comment)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index
data_dtm

,able,accidentally,acquiring,actions,aether,agents,agrees,allies,allow,america,...,wife,wilson,withstand,work,works,world,worthy,wrestles,years,york
index,,,,,,,,,,,,,,,,,,,,,
Comment 0,0,0,1,1,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
Comment 1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,0,0,0,2,0
Comment 2,0,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,1,0,0,0,0
Comment 3,0,1,0,0,0,1,0,0,1,0,...,0,0,0,0,1,0,0,0,0,1
Comment 4,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
Comment 5,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,1,1,0,0
Comment 6,0,0,0,0,0,0,0,0,0,1,...,0,2,0,0,0,0,0,0,0,0


In [58]:
from gensim import matutils, models
import scipy.sparse

In [59]:

id_word_dict={}
for i in range(489):
    id_word_dict[i]=data_dtm.columns[i]
print(id_word_dict)
tdm = data_dtm.transpose()

{0: 'able', 1: 'accidentally', 2: 'acquiring', 3: 'actions', 4: 'aether', 5: 'agents', 6: 'agrees', 7: 'allies', 8: 'allow', 9: 'america', 10: 'ancient', 11: 'appears', 12: 'arc', 13: 'archery', 14: 'army', 15: 'arrest', 16: 'artificial', 17: 'asgard', 18: 'assassin', 19: 'assault', 20: 'attacked', 21: 'attempt', 22: 'attempts', 23: 'avengers', 24: 'avoid', 25: 'banner', 26: 'barnes', 27: 'barton', 28: 'bartons', 29: 'battle', 30: 'beard', 31: 'begrudgingly', 32: 'best', 33: 'bested', 34: 'blasts', 35: 'board', 36: 'bound', 37: 'brief', 38: 'briefcase', 39: 'bring', 40: 'brought', 41: 'bruce', 42: 'brutally', 43: 'bucky', 44: 'burden', 45: 'captain', 46: 'cardiac', 47: 'carol', 48: 'cartels', 49: 'carter', 50: 'cassie', 51: 'causing', 52: 'chance', 53: 'child', 54: 'circuit', 55: 'clint', 56: 'clints', 57: 'collect', 58: 'concedes', 59: 'concern', 60: 'confirming', 61: 'confirms', 62: 'confiscate', 63: 'conflicted', 64: 'confront', 65: 'consciousness', 66: 'consciousnesses', 67: 'conso

In [60]:
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)

In [67]:
lda = models.LdaModel(corpus=corpus, id2word=id_word_dict, num_topics=5, passes=1000)
lda.print_topics()

[(0,
  '0.020*"time" + 0.020*"stark" + 0.020*"stone" + 0.018*"banner" + 0.015*"thanos" + 0.013*"space" + 0.013*"rogers" + 0.013*"stones" + 0.011*"plan" + 0.011*"lang"'),
 (1,
  '0.002*"romanoff" + 0.002*"rest" + 0.002*"earth" + 0.002*"nebula" + 0.002*"family" + 0.002*"rocket" + 0.002*"asgard" + 0.002*"universe" + 0.002*"did" + 0.002*"following"'),
 (2,
  '0.038*"stone" + 0.024*"nebula" + 0.015*"thor" + 0.015*"travel" + 0.015*"soul" + 0.010*"romanoff" + 0.010*"asgard" + 0.010*"earth" + 0.010*"rocket" + 0.010*"rhodes"'),
 (3,
  '0.022*"quantum" + 0.017*"realm" + 0.011*"romanoff" + 0.011*"snap" + 0.011*"time" + 0.011*"lang" + 0.011*"passed" + 0.011*"years" + 0.011*"rogers" + 0.011*"stark"'),
 (4,
  '0.035*"thanos" + 0.022*"stones" + 0.017*"stark" + 0.014*"avengers" + 0.014*"rogers" + 0.014*"gauntlet" + 0.011*"infinity" + 0.011*"past" + 0.009*"earth" + 0.009*"reunited"')]